## Dash Cytoscape

Dash Cytoscape - это компонент модуль для создания легко настраиваемых, высокопроизводительных, интерактивных и веб-визуализаций сетей. Он использует Cytoscape.js, хорошо интегрирован с макетами Dash и обратными вызовами.

Элемент (узел или связь) описывается словарем. Ключи:
* `data` - характеристики элемента (ID, Label, ...)
    * для ребер `data` должен содержать ключи `source` и `target`
* `position` - координаты (только для узлов)
* булевы атрибуты:
    * `locked` - True, если позицию узла нельзя менять
    * `selected` - True, если элемент выбран сразу после инициализации
    * `selectable` - True, если элемент может быть выбран
    * `grabbable` -  True, сли узел может быть захвачен и перемещен пользователем
* `classes` - класс или классы элемента (через пробел)

Подобно классам CSS, классы элементов используются для стилизации групп элементов с помощью селектора. Элементу можно присвоить  класс или несколько классов (разделенных пробелом).

In [22]:
%%file 06_dash_cytoscape/path.py
from dash import Dash, html, dcc
import dash_cytoscape as cyto
import networkx as nx
G = nx.karate_club_graph()
pos = nx.spring_layout(G, seed=42)

my_stylesheet = [
    # Стиль для узлов
    {
        'selector': 'node',
        'style': {
            'content': 'data(label)'
        }
    },
    # Стиль для классов
    {
        'selector': '.red',
        'style': {
            'background-color': 'red',
            'line-color': 'red'
        }
    },
    {
        'selector': '.triangle',
        'style': {
            'shape': 'triangle'
        }
    }
]


app = Dash(__name__)

app.layout = html.Div([
    cyto.Cytoscape(
        id='cytoscape-two-nodes',
        layout={'name': 'preset'}, # располагает в соответствии с координатами узлов
        style={'width': '100%', 'height': '800px'},
        stylesheet=my_stylesheet, # описание всех используемых классов
        elements=[
            {"data": {"id": 0, "label": "0 (locked)"}, "position": {"x": 0, "y": 0}, "locked": True, "classes": "red"},
            {"data": {"id": 1, "label": "1 (selected)"}, "position": {"x": 150, "y": 0}, "selected": True, "classes": "triangle"},
            {"data": {"id": 2, "label": "2 (!selectable)"}, "position": {"x": 250, "y": 0}, "selectable": False, "classes": "triangle"},
            {"data": {"id": 3, "label": "3 (!grabbable)"}, "position": {"x": 350, "y": 0}, "grabbable": False, "classes": "red"},
            {"data": {"source": 0, "target": 1}},
            {"data": {"source": 1, "target": 2}},
            {"data": {"source": 2, "target": 3}},
        ]
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting 06_dash_cytoscape/path.py


![Cyto Path](06_dash_cytoscape/cyto_path.png)

## Составные узлы

Составные узлы - это узлы, которые содержат (родительские) или содержатся (дочерние) внутри другого узла. Родительский узел не имеет ни позиции, ни размера, поскольку эти значения автоматически вычисляются на основе того, как настроены дочерние узлы.

In [21]:
%%file 06_dash_cytoscape/compound.py
from dash import Dash, html, dcc
import dash_cytoscape as cyto
import networkx as nx
G = nx.karate_club_graph()
pos = nx.spring_layout(G, seed=42)

app = Dash(__name__)

app.layout = html.Div([
    cyto.Cytoscape(
        id='cytoscape-two-nodes',
        layout={'name': 'preset'}, # располагает в соответствии с координатами узлов
        style={'width': '100%', 'height': '800px'},
        stylesheet=[ # описание всех используемых классов
            {
                'selector': 'node',
                'style': {
                    'content': 'data(label)'
                },
            },
            {
                'selector': '.countries',
                'style': {
                    'width': 5
                }
            },
            {
                'selector': '.cities',
                'style': {
                    'line-style': 'dashed'
                }
            },
        ], 
        elements=[
            # родительские узлы
            {"data": {"id": "rus", "label": "Россия"}},
            {"data": {"id": "br", "label": "Беларусь"}},
            # дочерние узлы
            {"data": {"id": "mos", "label": "Москва", "parent": "rus"}, "position": {"x": 100, "y": 100}},
            {"data": {"id": "dg", "label": "Долгопрудный", "parent": "rus"}, "position": {"x": 100, "y": 200}},
            {"data": {"id": "min", "label": "Минск", "parent": "br"}, "position": {"x": 400, "y": 200}},
            # связи
            {"data": {"source": "rus", "target": "br"}, "classes": "countries"},
            {"data": {"source": "dg", "target": "mos"}, "classes": "cities"},
            {"data": {"source": "min", "target": "mos"}, "classes": "cities"},
            
        ]
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting 06_dash_cytoscape/compound.py


![Cyto Compound](06_dash_cytoscape/cyto_compound.png)

## Макеты

Макет определяет, как узлы будут расположены на экране. При создании трейса `cyto.Cytoscape` нужно указать аргумент `layout` в виде словаря, в котором обязательно должен быть ключ `name`. Значение `name` может быть следующим ([документация](https://js.cytoscape.org/#layouts)):
* preset - расположение на основе заданных координат
* random - в произвольных положениях в окне просмотра
* grid - в виде сетки
* circle - в виде окружности
* concentric - в виде концентрических окружностей
* breathfirst - в виде иерархии (подход для деревьев)
* cose - на основе физ. симуляции.

Каждый из этих макетов имеет дополнительные аргументы, которые влияют на работу алгоритма (кол-во столбцов и строк для решетки, откуда начинать выполнять BFS и т.д.)


In [32]:
%%file 06_dash_cytoscape/layout.py
from dash import Dash, html, dcc
import dash_cytoscape as cyto
import networkx as nx
import math

def city_graph():
    nodes = [
        {
            'data': {'id': short, 'label': label},
            'position': {'x': 20 * lat, 'y': -20 * long}
        }
        for short, label, long, lat in (
            ('la', 'Los Angeles', 34.03, -118.25),
            ('nyc', 'New York', 40.71, -74),
            ('to', 'Toronto', 43.65, -79.38),
            ('mtl', 'Montreal', 45.50, -73.57),
            ('van', 'Vancouver', 49.28, -123.12),
            ('chi', 'Chicago', 41.88, -87.63),
            ('bos', 'Boston', 42.36, -71.06),
            ('hou', 'Houston', 29.76, -95.37)
        )
    ]

    edges = [
        {'data': {'source': source, 'target': target}}
        for source, target in (
            ('van', 'la'),
            ('la', 'chi'),
            ('hou', 'chi'),
            ('to', 'mtl'),
            ('mtl', 'bos'),
            ('nyc', 'bos'),
            ('to', 'hou'),
            ('to', 'nyc'),
            ('la', 'nyc'),
            ('nyc', 'bos')
        )
    ]
    elements = nodes + edges
    return elements


app = Dash(
    __name__,
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
)
elements = city_graph()

app.layout = html.Div([
        html.Div(
            [
                html.Div([
                    'cytoscape-grid',
                    cyto.Cytoscape(
                        id='cytoscape-grid',
                        elements=elements,
                        style={'width': '100%', 'height': '350px'},
                        layout={
                            'name': 'grid',
                            'rows': 3
                        }
                    )
                ], className="four columns"),
                html.Div([
                    'cytoscape-circle',
                    cyto.Cytoscape(
                        id='cytoscape-circle',
                        elements=elements,
                        style={'width': '100%', 'height': '350px'},
                        layout={
                            'name': 'circle',
                            'radius': 250,
                            'startAngle': math.pi * 1 / 6,
                            'sweep': math.pi * 2 / 3
                        }
                    )
                ], className="four columns"),
                html.Div([
                    'cytoscape-breadthfirst1',
                    cyto.Cytoscape(
                        id='cytoscape-breadthfirst1',
                        elements=elements,
                        style={'width': '100%', 'height': '350px'},
                        layout={
                            'name': 'breadthfirst',
                            'roots': '[id = "nyc"]' # специальный синтаксис для указания ID узлов 
                        }
                    )
                ], className="four columns"),
            ],
            className='Row'
        ),
        html.Div(
            [
                html.Div([
                    'cytoscape-breadthfirst2',
                    cyto.Cytoscape(
                        id='cytoscape-breadthfirst2',
                        elements=elements,
                        style={'width': '100%', 'height': '350px'},
                        layout={
                            'name': 'breadthfirst',
                            'roots': '#van, #mtl' # специальный синтаксис для указания ID узлов 
                        }
                    )
                ], className="four columns"),
                html.Div([
                    'cytoscape-cose',
                    cyto.Cytoscape(
                        id='cytoscape-cose',
                        elements=elements,
                        style={'width': '100%', 'height': '350px'},
                        layout={
                            'name': 'cose'
                        }
                    )
                ], className="four columns"),
            ],
            className="Row"
        )
    ]
)

if __name__ == '__main__':
    app.run_server(debug=True)


Overwriting 06_dash_cytoscape/layout.py


![Cyto Layout](06_dash_cytoscape/cyto_layout.png)